In [1]:
! pip install pip --upgrade
! pip install pyconll
! pip install transformers
! pip install dotmap
! pip install spacy
# ! pip install torch

In [2]:
import logging
import os
from dotmap import DotMap
os.environ["CUDA_VISIBLE_DEVICES"] ="0"
import torch
from transformers import (RobertaConfig, RobertaModel, RobertaTokenizer)
from transformers import AutoTokenizer
import nltk.data
nltk.download('punkt')

from predict import Predict
logger = logging.getLogger(__name__)

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
args = {
    "loadmodel_name_path": "trained_models/scoreNN_PROP_NER_rand_conll_GMB_tpkoc",
        
    "local_rank":-1,
    "n_gpu":0,
    "device":"cuda" if torch.cuda.is_available() else "cpu",
    "do_lower_case":True,
    "per_gpu_eval_batch_size": 1,
    "max_seq_length":192,

    "output_dir":"outputs/",
    
    "minimum_score_to_display":0.2,
    "divisions":16,
    "categories" :["","NOUN_","PROPN_","NER_"],
    

    "seed": 50
}
args = DotMap(args)
print("set arguments")

set arguments


In [4]:
doc1 = """Johnson said the UK would work to unite the international community behind a "clear plan for dealing with the Taliban." The Prime Minister, who is the current president of the Group of Seven leading nations, said he was looking to convene a meeting of leaders in coming days.

"We are clear, and we have agreed that it'd be a mistake for any country to recognize any new regime in Kabul prematurely or bilaterally," Johnson, who spoke with US President Joe Biden and other world leaders in recent days, said.

"We will judge this regime on the choices it makes and by its actions rather than its words," he added.

The refugee plan, which is similar to a similar package for Syria in 2015, came under immediate attack from lawmakers, who said it fell short of what was required, both in terms of speed and numbers.

"The government has said 5,000 will be brought to resettle in the UK this year," Chris Bryant , a parliament member from the main opposition Labour Party , said. "What are the other 15,000 meant to do? Hang around and wait to be executed?"

Johnson said British officials were doing all they can to evacuate UK and Afghan citizens who helped the British forces based in Afghanistan and that the Taliban have not sought to disrupt the operation.

"The situation has stabilized since the weekend, but it remains precarious, and the UK officials on the ground are doing everything that they can to expedite the movement of people," he said. "At the moment, it would be fair to say that the Taliban are allowing that evacuation to go ahead."

Johnson told Parliament that events in Afghanistan have "unfolded faster than even the Taliban predicted," but the Prime Minister denied that his government had been caught unawares.

Labour Party leader Keir Starmer said the Conservative government had to take its share of the responsibility for the crisis in Afghanistan.

"There's been a major miscalculation of the resilience of the Afghan forces and a staggering complacency from our government about the Taliban threat," he said.

Like Biden, Johnson is facing criticism over Britain's hasty retreat from Afghanistan and its chaotic evacuation of British citizens and the thousands of Afghans it has employed over the past two decades. Criticism has been particularly acute from veterans and the families of the 457 British troops who died in the country while fighting there as part of the US-led Nato military operation.

Demonstrations are planned outside of Parliament to call for support for Afghans and their families."""

#############################################################################################
#############################################################################################
doc2 = """US President Joe Biden WASHINGTON: The Taliban must decide whether it wants to be recognised by the international community, US President Joe Biden said in an ABC interview aired on Thursday, adding that he did not think the group had changed its fundamental beliefs.

Asked if he thought the Taliban had changed, Biden told ABC News , "No."

"I think they're going through a sort of existential crisis about: Do they want to be recognised by the international community as being a legitimate government? I'm not sure they do," he said, adding that the group appeared more committed to its beliefs.

But, he said, the Taliban also had to grapple with whether it could provide for Afghans.

"They also care about whether they have food to eat, whether they have an income that... can run an economy, they care about whether or not they can hold together the society that they in fact say they care so much about," Biden said in the interview, taped on Wednesday. "I'm not counting on any of that."

He added that it would take economic and diplomatic pressure -- not military force -- to ensure women's rights.
"""
#############################################################################################
#############################################################################################
doc3 = """A view of Beijing Lize Financial Business District. [Photo provided to China Daily] Beijing plans to invest 590 billion yuan ($91.3 billion) between this year and 2025 to develop its southern districts, which have already undergone massive changes in the last 10 years.

The plan includes 260 major projects covering education, healthcare, business and public welfare sectors, said Li Sufang, deputy director of the Beijing Municipal Development and Reform Commission, at a news conference on July 29.

The rapid development of southern Beijing began in 2010 when the first round of the ongoing action plan was launched.

Lan Tian, who is in his 30s and lives in southern Beijing's Daxing district, said he never expected that his residential community would develop so quickly, with shopping malls, convenience stores and public parks multiplying.

"I feel that my life is becoming more colorful due to the improvements in recent years," he said. "A big surprise was the completion of Daxing International Airport, which has shortened travel time to the airport for business trips and other travel."

"I like the airport. It's spacious, clean and modern. I usually go half a day earlier to enjoy the restaurants, bookstores and shopping in the terminal," Lan added.

During the fourth round of the action plan for southern Beijing, one focus will be on high-quality development around the airport, aimed at promoting the area as a new driver of economic growth in the capital.

In Daxing district's Lixian township, the Beijing Free Trade Zone's innovation service center stands ready. Dozens of companies have moved in and are busy decorating their offices for opening.

The companies work mainly in biological medicine and finance, attracted by the center's policies that support international talent and offer convenient market supervision.

In addition to Daxing, Beijing's southern region includes three other districts: Fengtai, Fangshan and the Beijing Economic Technological Development Area.

Liu Li, deputy head of the development area, said it will strengthen cooperation with southern Beijing's other districts in industries like smart manufacturing, intelligent new energy automobiles and logistics.

Zhou Xinchun, deputy head of Fengtai, said the district's Lize area, which is focused on the financial industry, will develop quickly in the next five years.

"Transportation and power supplies in the district will both improve," he said.

Lyu Chenfei, deputy head of Fangshan, said the district will focus on culture and tourism over the next five years through a number of key projects including a national panda breeding center, a wine country and a Lego theme park.

According to official data, GDP in the capital's southern region has grown from 229.6 billion yuan to 559.2 billion yuan over the last 10 years.

Under the current phase of the action plan, which will be carried out over the next five years, the region aims to build four more 100-billion-yuan industrial clusters, covering the sectors of biotechnology and health, the new generation and innovative use of information technology, robotics and high-end equipment.

Major new cultural facilities will also be built, including clusters of museums and an art center for the National Ballet of China, and new schools and hospitals will be added. The plans include room for environmental advancement and future adjustments.

In addition, Beijing will also support the southern districts in creating more than 10,000 hectares of green space in the next five years through the renovation and construction of major wetlands and forest parks as part of its effort to become a national forest city."""
#############################################################################################
#############################################################################################
doc4 = """Bitcoin prices have been on a wild ride this year. While prices for the bellwether cryptocurrency more than doubled to levels of around $65,000 between early January and mid-April 2021, driven by higher institutional interest, prices fell back to levels of just under $30,000 as of mid-July, due to China’s crackdown on Bitcoin trading and the U.S. Federal Reserve’s increasingly hawkish stance. However, the currency has recovered swiftly from the recent lows, rising by about 65% since July 20th, and trades at about $50,000 currently. The recent recovery is apparently driven by factors including short covering, as well as signs that the U.S. SEC was increasingly open to allowing Bitcoin exchange-traded funds.

Investors who want to play the long-term gains in cryptos, without exposing themselves to the volatility and boom and bust cycles in cryptocurrency pricing, should take a look at our indicative theme on Cryptocurrency Stocks which includes semiconductors, payments, and brokerage companies that have exposure to the cryptocurrency value chain. The theme has gained roughly 24% year-to-date, compared to the S&P 500 which is up by about 19% over the same period. Within our theme, graphics processor major Nvidia has fared the best, with its stock rising by 59% year-to-date, driven by strong demand for its chips as well as its recent stock split. On the other side, exchange major CME Group has been the worst performer in our theme, rising by just about 9% over the same period.

[7/1/2021] Bitcoin Prices Crash

Bitcoin prices have crashed from levels of around $62,000 in mid-April to just about $34,000 as of Wednesday. The crypto bear market appears to be driven by a host of factors, including China’s crackdown on Bitcoin trading and mining and Tesla’s TSLA unexpected reversal of its decision on accepting the digital currency as payment for its cars. Moreover, the U.S. Federal Reserve has turned increasingly hawkish following its mid-June meeting indicating that it could start hiking interest rates from 2023, rather than 2024. This is also likely putting pressure on non-productive assets such as cryptocurrency.

Now, although the market sentiment for cryptos, in general, is clearly bearish, cryptocurrencies have the potential to be one of the most disruptive technologies of our time. Moreover, with increasing institutional interest, they appear to be here to stay as an asset class. Investors who want to play the long-term gains in cryptos, without exposing themselves to the volatility and boom and bust cycles in cryptocurrency pricing, should take a look at our indicative theme on Cryptocurrency Stocks which includes semiconductors, payments, and brokerage companies that have exposure to the cryptocurrency value chain. The theme has gained roughly 22% year-to-date, compared to the Nasdaq-100, which has returned about 15% over the same period. Within our theme, graphics processor major Nvidia has fared the best, with its stock rising by 53% year-to-date, driven by strong demand for its chips as well as its planned stock split. On the other side, processor major Advanced Micro Devices has been the worst performer, with its stock up by just about 2% this year.

[4/5/2021] Crypto Stocks To Watch

Cryptocurrency prices have surged this year. Crypto bellwether Bitcoin has almost doubled year-to-date to levels of about $60k as of the end of last week, as more institutional investors warm up to the currency, with companies such as Tesla also indicating that they will accept bitcoin payments from customers. However, investing in Bitcoin and other cryptos is risky at current levels. With Covid-19 cases on the decline and vaccination rates in the U.S. rising, the economic outlook is improving. Bond yields are also trending higher, with the yield on the 10-year Treasury note rising from around 1% earlier this year to close to 1.70% currently. Considering this, investors are moving funds back to real-economy sectors, and it’s likely that non-productive assets such as Bitcoin will eventually be impacted. Our indicative theme on Cryptocurrency Stocks could be a safer way to play the long-term upside from cryptocurrencies, without taking a position in individual currencies. The theme, which includes semiconductor, payments, and brokerage companies that have exposure to the cryptocurrency value chain, is up by about 122% since 12/31/2019, compared to the S&P 500 which is up by about 24%. Here is a look at the recent developments for some of the stocks in our theme.

MORE FOR YOU Toyota, Ford, VW Cut Production On Chip Shortages. These Stocks Stand To Benefit Enjin Coin: All About Fake Swords, Fake Real Estate, And NFTs Near Future Another Covid Lockdown Would Break The U.S. Economy PayPal , a large online payments processor, allows customers to buy, hold, and sell cryptocurrencies. The company has taken this a step further, launching its “Checkout with Crypto” service last week, enabling U.S. customers to use their crypto holdings to pay at millions of online merchants worldwide.

Nvidia indicated a few weeks ago, that it would be launching a GPU that was dedicated to mining cryptocurrencies. Nvidia has seen gaming GPU shortages in the past, as its gaming processors were being used to mine cryptocurrency. The move should let the company better target the crypto mining space, while potentially alleviating GPU shortages for gaming.

CME Group , a derivatives exchange, plans to launch new Micro Bitcoin futures from early May, subject to regulatory approval. The smaller-sized futures contracts are a tenth of the size of one Bitcoin and will be targeted at institutions and sophisticated traders.

[3/16/2021] Crypto Stocks To Watch As Bitcoin Nears $55k

Cryptocurrency prices have surged this year, with Bitcoin now up by around 90% since early January, trading at levels of around $55k currently. However, Bitcoin looks quite risky at current levels due to a couple of factors. With Covid-19 cases on the decline, the economic outlook is improving and bond yields are also on the rise. Considering this, investors are likely to start moving funds back to real-economy sectors, and non-productive assets such as Bitcoin, which rallied big through the pandemic, could be impacted. Our indicative theme on Cryptocurrency Stocks could be a safer way to play the long-term upside from cryptocurrencies, without taking a position in individual currencies. The theme, which includes semiconductor, payments, and brokerage companies that have exposure to the cryptocurrency value chain, is up by about 124% since 12/31/2019, compared to the S&P 500 which is up by about 22%. Year-to-date, the theme is up by 4%, compared to the S&P 500 which is up around 5%. Within our theme, digital payments player Square SQ has been the strongest performer, rising by 287% since the end of 2019, while exchange major CME Group has been the worst performer in our theme, rising by just about 4% over the same period.

[2/18/2021] Which Stocks Should You Buy As Bitcoin Hits $50k?

Cryptocurrency have continued their surge this year, driven by retail and institutional interest. While the crypto market bellwether Bitcoin has soared 5x over the last 12 months and by almost 80% year-to-date, lesser-known crypto Dogecoin is up by over 10x year-to-date. To be sure, these currencies remain a risky investment, given their relative lack of utility and extremely high volatility. Our indicative theme on Cryptocurrency Stocks – which includes semiconductor, payments, and brokerage companies that have exposure to the cryptocurrency value chain - could be a safer way to play the crypto space, without taking a position in individual currencies. The theme has returned about 151% since the end of 2019, compared to about 22% for the S&P 500. The theme is also up 14% year-to-date, versus about 5% for the S&P 500. Payments player Square remains the strongest performer in our theme, rising by over 330% since the end of 2019, driven by its Bitcoin exposure, and also by an increasing preference for digital payments among consumers and businesses. On the other side, financial exchange major CME Group has been the worst performer in our theme, down by about -5% over the same period.

[Updated 1/20/2021] Cryptocurrency Stocks

Bitcoin prices have surged by more than 50% over the last month and remain up by about 4x over the 12 months, due to higher retail and institutional interest. That being said, Bitcoin remains a risky investment, given its relative lack of utility and extremely high volatility. Our indicative theme on Cryptocurrency Stocks – which includes semiconductor, payments, and brokerage companies that have some exposure to the cryptocurrency value chain - could be a safer way to play the crypto space, without betting on individual currencies. The theme has done well, rallying by over 100% over the last year. Below is a bit more about the key companies in the theme and how they have been faring.

Nvidia stock saw big gains over 2020, driven by growing demand for its GPUs from gaming consoles and data centers and its planned acquisition of CPU designer ARM. The company recently indicated that it could restart the production of dedicated crypto mining GPUs, which essentially remove the video outputs that are required for gaming.

Square a payments company, allows users to buy and sell bitcoin via its Cash mobile payment application. The company said that bitcoin-related Revenue surged by about 8x over the first nine months of 2020 to around $2.8 billion.

PayPal a large online payments processor has also reported increasing engagement on its PayPay mobile app after it introduced support for bitcoin last October, with about half its crypto users opening the PayPal app daily.

CME Group the world’s largest financial derivatives exchange, also offers bitcoin futures contracts. The exchange has emerged as the world’s largest futures exchange for Bitcoin, with open interest - contracts traded but not squared off - for the cryptocurrency standing at $2.1 billion.

[Updated 12/7/2020] How Nvidia, Square & PayPal Stocks Are Benefiting From Soaring Bitcoin Prices

Bitcoin prices have surged by over 160% year-to-date, driven by multiple factors including higher institutional interest, fintech companies Paypal PYPL and Square’s moves into the crypto space, and also by a view that scarce digital currency could be a hedge against inflation and a weakening U.S. dollar. Our indicative theme on Cryptocurrency Stocks – which includes semiconductor, payments, and brokerage companies that have some exposure to the cryptocurrency space – is up a solid 110% year-to-date, compared to the S&P 500 which is up by just about 14% over the same period. Key stocks in our theme include Square, PayPal Holdings , Nvidia , and CME Group .

Although crypto likely represents a small portion of these company’s revenues at the moment, it does have the potential to be big. For example, Payment apps Square and PayPal, who make money by buying bitcoin from brokers and selling them to their customers by adding a “spread” stand to benefit as prices and volumes rise. Square’s Cash App generated about $1.63 billion in Bitcoin-related sales in Q3 2020, an 11x jump year-over-year. Similarly, Nvidia also benefits from demand for its top-of-the-line GPUs such as the Ampere lineup from cryptocurrency miners.

[Updated 10/29/2020] Cryptocurrency Stocks To Watch

Interest in cryptocurrency is surging once again with Bitcoin prices up by almost 30% over the last month, driven by growing institutional interest and Paypal’s recent move to allow its customers to buy and sell certain cryptocurrencies. Our indicative theme on Cryptocurrency Stocks – which includes semiconductor, payments, and brokerage companies that have some exposure to the cryptocurrency space – is up 88% year-to-date, compared to the S&P 500 which is up by just about 5% over the same period. This theme could be of interest to investors who are looking for upside from cryptocurrency adoption and prices, but want to avoid buying into the currencies themselves considering the volatility, risk of fraud, or cyber theft, or potential portfolio mandates. Below is a bit more about the stocks in our theme and how they have fared this year.

Square cash mobile wallet app has emerged as a very popular way for people to buy and sell Bitcoin. The stock has gained a whopping 183% year-to-date, as investors bet that Square’s digital payment solutions will continue to gain traction through and post Covid-19. The stock has dropped by about -9% over the last 5 trading days, however.

Nvidia a semiconductor company that is best known for its graphic processing units which are increasingly used in machine learning and AI, is also finding applications in bitcoin mining. The stock has gained over 131% year to date although it declined by about -3% over the last week.

PayPal a large online payments processor, has indicated that it would allow customers to buy, hold, and sell cryptocurrencies such as Bitcoin and Ethereum directly within its PayPal and Venmo app. The stock is up by 88% this year and was down by about -1% over the last 5 trading days.

Advanced Micro Devices is a semiconductor major that makes CPUs and GPUs, which are increasingly used in bitcoin mining. The stock has gained about 79% year-to-date and remained largely flat over the last 5 trading days.

CME Group the world’s largest financial derivatives exchange, also offers bitcoin futures contracts. The stock is down by about -18% year-to-date and declined by about -2% over the last week."""

doc5 = """[indianexpress.com.] August 23, 2021 7:27:38 pm Written by Anubha Gupta Occupying the presidential palace in Kabul, the triumphant Taliban leaders with bazookas in their hands proclaimed that “War has ended in Afghanistan”. Yet, from all around Afghanistan, we see images of anxiety, desperation, chaos, and fear. Winter has finally arrived in Afghanistan. The situation demands grappling with some serious questions: How has the Taliban regained territorial control over Afghanistan? How has the Taliban managed to be so strong, notwithstanding efforts by America to bring it down? What has made this possible? Among many causal explanations, the geopolitical economy of opium can be traced as a plausible factor that led to the Taliban’s victory. Post industrialisation, many political geographers including Mackinder and Spykman established the relationship among space, resources, and politics. There is an interrelationship between resources and war. A resource war is “an armed conflict in which the control and revenue of natural resources are significantly involved in the economy of the conflict or the motivations of the belligerents”. In the 1990s, there were many key resources involved in the war economy — cocaine in Columbia, diamond in DR Congo, oil in Algeria, etc. A variety of actors, from local entrepreneurs and international brokers to global contraband networks and major transnational corporations are involved in the resource war. At every stage in the tragic history of wars in Afghanistan — the Great Game, the covert war of the 1980s, the civil war of the 1990s, and the protracted war post-9/11 — opium has played a central role in shaping the country’s destiny. Opium cultivation was minimal in Afghanistan before the Great Game. The commercialisation of opium started with the coming of the British. It was during the Cold War that a full-blown impetus was provided to the cultivation of opium by the US, which backed the mujahideen to expel the Soviet Union. The Central Intelligence Agency, for the next 10 years, provided the mujahid guerrillas with an estimated $3 billion in arms. The reason most cited for the success of the US in expelling the USSR from Afghanistan is the willy-nilly continuation of drug trafficking by US-backed mujahids. During the 1980s, the Afghan-Pakistan border was converted into a springboard for the international heroin trade. The US state department reported in 1986 that in tribal areas “there is no police force. There are no courts. There is no taxation. No weapon is illegal, hashish and opium are often on display.” Soon, the region became the world’s largest heroin producer. Today, Afghanistan has become the world’s first true narco-state – a country where illicit drugs define political choices, dominate the economy, and determine the fate of foreign interventions. After capturing Kabul in 1996, the Taliban encouraged local opium cultivation by offering government protection to the export trade and also collecting much-needed taxes. The United Nations’ opium surveys showed that during the Taliban’s first three years in power, Afghanistan’s opium crop accounted for 75 per cent of world production. In a war-ravaged Afghanistan, opium fields were a blessing in disguise for the poor as its cultivation required nine times more labour than Afghanistan’s staple crop, wheat. Thus, opium has been intricately woven into the fabric of conflict in Afghanistan. Being a distant decentralised resource, it became an essential part of the local economy, which was then controlled by a new class of warlords. Post 9/11, America only targeted the guerrillas and not the drugs, thus sowing the seeds for the revival of the Taliban. Afghan heroin is trafficked to many destinations worldwide. Europe is the largest market for Afghan heroin. South-East Asia has also become a significant destination. In the past few years, Africa has also received Afghan heroin flows and is becoming a cost-effective trafficking route to North America, Europe and island countries. The opium trade has a network of people from farmers and makeshift lab operators to businessmen working through intermediaries and MNCs. Helmand is the biggest opium-producing province in Afghanistan. In a map released by the UN in October 2015, the revived Taliban was reported to be most active and threatening in Helmand province. A UN Security Council investigation found that the Taliban had systematically tapped “into the supply chain at each stage of the narcotics trade”. According to the New York Times, it “has become difficult to distinguish the group from a dedicated drug cartel”. It was also reported that opium is supplying 60 per cent of the Taliban’s funds for weapons and wages. This significant illegal commodity was then accorded secrecy from fields to foreign markets, inducing corruption. The Afghan war has been a prolonged resource war riding on the back of illicit abundant opium cultivation in the hinterland by the local warlords. Opium has shaped the nature and course of the war over the years. Thus, the future of Afghanistan and its transition to peace will be decided by the very same opium. The writer is a research scholar at the Centre for International Politics, Organisations and Disarmament at Jawaharlal Nehru University, New Delhi 📣 The Indian Express is now on Telegram. Click here to join our channel (@indianexpress) and stay updated with the latest headlines For all the latest Opinion News , download Indian Express App. - - The Indian Express website has been rated GREEN for its credibility and trustworthiness by Newsguard, a global service that rates news sources for their journalistic standards. - [indianexpress.com.]"""
doc6 = """Heroin production has boomed in Afghanistan in recent years, helping fund the Taliban, and experts say they will struggle to wean themselves off the profitable trade despite their promise to do so. Speaking at its first press conference since taking power this week, Taliban spokesman Zabihullah Mujahid promised that the new government would not turn the world’s leading producer of opium into a fully-fledged narcostate. “We are assuring our countrymen and women and the international community, we will not have any narcotics produced,” Mujahid told reporters in Kabul. “From now on, nobody’s going to get involved [in the heroin trade], nobody can be involved in drug smuggling.” It may take some doing, especially if the new government does not have the same access to financial reserves and foreign aid that have sustained Afghanistan’s fragile economy for two decades. Some major donors have already halted support for the country, one of the world’s poorest, including the IMF which on Wednesday announced it would freeze aid to the country amid uncertainty over recognition of the government. Taliban seized US military biometric devices, report says In 2020, aid flows represented 42.9 per cent of Afghanistan’s US$19.8 billion GDP, according to World Bank data. Beyond cash on hand, the Taliban has few other funds to draw upon – most of the country’s reserves are held overseas. The United States this week froze roughly US$9 billion in assets belonging to Da Afghan Bank, or DAB, the nation’s central bank. About US$7 billion of that is with the Federal Reserve Bank of New York, according to DAB’s top official, Ajmal Ahmady, who fled the capital Kabul as it fell to the Taliban. The Biden administration has also stopped shipments of cash. The Taliban remains on the US Treasury’s sanctions designation list. The vast majority of the world’s opium and heroin comes from Afghanistan, with production and exports centred in areas controlled by the Taliban, which has taxed the drugs heavily during their 20-year insurgency. It has become a key resource for the group and they could struggle to ban it, said Jonathan Goodhand, an expert in the international drugs trade at SOAS University of London. “Drugs will bring out a set of tensions in the movement,” he predicted. On one hand, “they want to create this image of themselves as more moderate and more open to engagement with the West and they realise drugs is one of way of doing this,” he said. But on the other, any repression would hit farmers in the Taliban political heartlands of Helmand and Kandahar provinces in particular. The Afghanistan images that could tarnish Joe Biden’s legacy “It’s going to struggle to take a very aggressive approach to drugs,” he added. UN officials reported that the Taliban likely earned more than US$400 million between 2018 and 2019 from the drug trade. A May 2021 US Special Inspector General for Afghanistan (SIGAR) report quoted a US official as estimating they derive up to 60 per cent of their annual revenue from illicit narcotics. Extortion of businesses as well as ransom from kidnapping also provide income, according to a 2020 report from a UN Security Council sanctions committee, which estimated the group’s revenues at $300 million to US$1.5 billion a year. In his premier press conference, Mujahid pleaded for “international assistance” to provide farmers with alternative crops to poppies, the source of sap that is refined into morphine and heroin. The appeal for international aid might prompt hollow laughs from people who worked in the coalition of Nato forces, NGOs and UN workers over the last 10 years that tried in vain to break Afghanistan’s reliance on poppy farming. The United States spent around US$8.6 billion from 2002 to 2017 in its doomed effort to combat the drugs trade, according to a 2018 SIGAR report. Those efforts included paying farmers to grow wheat or saffron, investing in transport links, as well as spraying defoliants on crops and bombing refining facilities. As Taliban takes over, some swap AK-47s for US-made rifles At each step, they found themselves thwarted by Taliban fighters who controlled the main poppy-growing regions and derived hundreds of millions of dollars from the industry, according to US and Afghan government estimates. Farmers in Taliban-controlled areas would often come under pressure to plant poppies from local warlords and fighters, investigations have found. As a result, the country has a near-monopoly on opium and heroin, accounting for 80 to 90 per cent of global output, according to the United Nations Office on Drugs and Crime (UNODC). The amount of land planted with poppies hit a record high in 2017 and has averaged around 250,000 hectares in the last four years, roughly four times the level of the mid-1990s, UN figures show. The narcotics policy of the new government will affect global heroin prices, with repercussions for Western countries and their addicts, as well as Russia, Iran, Pakistan and China – all major smuggling routes but also huge markets for Afghan drugs. In recent years, traffickers have also discovered that a plant commonly found in Afghanistan called ephedra can be used to create a key component of methamphetamine, better known as “crystal meth”. Still, spokesman Mujahid vowed Afghanistan would be a “narcotics-free country” moving forward. It’s not the first time the fundamentalist group has vowed to outlaw the trade. Production was banned in 2000, just before the group was overthrown by US-led forces. Profits and poppies: Afghanistan’s illegal drug trade a boon for Taliban Gretchen Peters, the American author of the book Seeds of Terror: How Heroin Is Bankrolling the Taliban and Al-Qaeda , said the Taliban’s previous ban on poppy cultivation was tactical. “They were under immense international pressure,” she said. “It was a ploy because they had so much stored up. They made a huge amount of money once the price shot up by 10 times.” “They are not going to get rid of the drug trade because they are too tied up with it.” “Afghanistan cannot survive without opium. It is simultaneously killing Afghanistan while also keeping a huge number of people alive,” she said, referring to the income the industry provides to poor farmers. Being in control of the country will offer the Taliban access to airlines, the state bureaucracy and banks which could be used to facilitate drug smuggling and money laundering, she explained. “I have no doubt they will exploit it.” Additional reporting by Reuters and Bloomberg"""

In [8]:
#######################################
MAIN_DOC = doc2
args.minimum_score_to_display=0.2
print("Predicting!\n"+"*"*75)
#######################################
p = Predict(args)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
inputs = tokenizer.tokenize(MAIN_DOC)
for sentence in inputs:
    if sentence not in [""]:
        words = nltk.word_tokenize(sentence)
#         words = sentence.split(" ")
        inp = p.load_input(words)
        p.run_batch_selection_eval(inp)

Predicting!
***************************************************************************
